# Pipeline labelisation

## Imports des packages

In [13]:
import sys
sys.path.append('../src')
import shutil
import yaml
import py7zr
import os
import s3fs
from satellite_image import SatelliteImage
from labeler import RILLabeler
from labeler import BDTOPOLabeler
from labeled_satellite_image import SegmentationLabeledSatelliteImage
from utils import *
from filter import *
import matplotlib.pyplot as plt
import numpy as np
import rasterio.plot as rp
import re
from datetime import datetime
from tqdm import tqdm

In [14]:
update_storage_access()

## Téléchargement des données Pléiades

In [15]:
environment = get_environment()

bucket = environment["bucket"]
path_s3_pleiades_data = environment["sources"]["PLEIADES"]
path_s3_bdtopo_data = environment["sources"]["BDTOPO"][2022]["guyane"]
path_local_pleiades_data = environment["local-path"]["PLEIADES"]
path_local_bdtopo_data = environment["local-path"]["BDTOPO"][2022]["guyane"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

On télécharge les fichiers depuis s3 (à mettre dans le setup.sh?)

In [16]:
fs.download(
        rpath=f"{bucket}/{path_s3_pleiades_data}",
        lpath=f"../{path_local_pleiades_data}",
        recursive=True)

In [17]:
fs.download(
        rpath=f"{bucket}/{path_s3_bdtopo_data}",
        lpath=f"../{path_local_bdtopo_data}",
        recursive=True)

## Découpage et filtrage des images

In [18]:
images_paths = [f"../{path_local_pleiades_data}/16bits/ORT_2022072050325085_U22N/" + p for p in os.listdir(f"../{path_local_pleiades_data}/16bits/ORT_2022072050325085_U22N/")]

In [19]:
date = datetime.strptime(re.search(r'ORT_(\d{8})', images_paths[0]).group(1), '%Y%m%d')

On importes les `SatelliteImage` dans une liste.

In [20]:
list_images = [
     SatelliteImage.from_raster(
        filename,
        date = datetime.strptime(re.search(r'ORT_(\d{8})', filename).group(1), '%Y%m%d'), 
        n_bands = 4
    ) for filename in tqdm(images_paths)]

On découpe les images initialement de dimension 2000 en plusieurs image de dimension `dim`. On ne garde ensuite que les images qui sont pertinentes à l'aide de la méthode `filter_images`. Une image contenant moins de 50% de pixels non noirs n'est pas retenue.

In [21]:
dim = 250

In [22]:
splitted_list_images = [im for sublist in tqdm(list_images) for im in sublist.split(250) if not is_too_black(im)]

In [23]:
len(splitted_list_images)

On peut représenter graphiquement nos images grâce à la méthode `plot` :

In [24]:
splitted_list_images[2796].plot([0, 1, 2]) # RGB (3 premières bandes)

On peut représenter l'image infrarouge : 

In [25]:
splitted_list_images[2796].plot([3, 0, 1]) # RGB (Infrarouge, rouge, vert)

Ou bien qu'une seule bande :

In [26]:
splitted_list_images[2796].plot([0]) # Une bande

## Labélisation avec le RIL

On définit un `labeler` incluant la taille des buffers que l'on souhaite ainsi que la forme de ceux-ci.

In [27]:
labeler_RIL = RILLabeler(date, buffer_size=6, cap_style=3)

A l'aide de notre `labeler` on créer une `SegmentationLabeledSatelliteImage`.

In [28]:
list_labeled_images = [
     SegmentationLabeledSatelliteImage(
        sat_im,
        labeler_RIL.create_segmentation_label(sat_im),
        "RIL",
        date
     ) for sat_im in tqdm(splitted_list_images)]

Il est également possible de représenter graphiquement l'image ainsi que son label grâce à la méthode `plot()`. Ici un exemple d'image ou l'on constate la présence d'immeubles : 

In [29]:
list_labeled_images[2796].plot([0,1,2])

Et voici un exemple sans immeuble : 

In [30]:
list_labeled_images[0].plot([0,1,2])

## Labélisation avec la BDTOPO

On peut réaliser la labelisation grâce à la BDTOPO plutôt que le RIL afin d'avoir des contours plus précis.

In [31]:
labeler_BDTOPO = BDTOPOLabeler(date)

In [32]:
list_labeled_images = [
     SegmentationLabeledSatelliteImage(
        sat_im,
        labeler_BDTOPO.create_segmentation_label(sat_im),
        "BDTOPO",
        date
     ) for sat_im in tqdm(splitted_list_images)]

In [33]:
list_labeled_images[2796].plot([0,1,2])

In [34]:
list_labeled_images[0].plot([0,1,2])